In [26]:
import pandas as pd
import copy
import heapq

In [27]:
class Eight_Puzzle_Problem:
    def __init__(self, initial_state):
        self.INITIAL_STATE = initial_state
        self.GOAL_STATE = [[1, 2, 3],
                           [4, 5, 6],
                           [7, 8, 0]]
        self.OPERATORS = ['up', 'down', 'left', 'right']

    def GOAL_TEST(self, state):
        return state == self.GOAL_STATE

class Node:
    def __init__(self, state, parent=None, action=None, g=0, h=0):
        self.STATE = state
        self.PARENT = parent
        self.ACTION = action
        self.g = g  # Cost from start
        self.h = h  # Heuristic estimate
        self.f = g + h

    def __lt__(self, other):
        return self.f < other.f

def EXPAND(node, operators):
    expanded_nodes = []
    # Find blank/zero position
    r, c = -1, -1
    for i in range(3):
        for j in range(3):
            if node.STATE[i][j] == 0:
                r, c = i, j
                break
    
    for op in operators:
        new_state = copy.deepcopy(node.STATE)
        if op == 'up' and r > 0:
            new_state[r][c], new_state[r-1][c] = new_state[r-1][c], new_state[r][c]
        elif op == 'down' and r < 2:
            new_state[r][c], new_state[r+1][c] = new_state[r+1][c], new_state[r][c]
        elif op == 'left' and c > 0:
            new_state[r][c], new_state[r][c-1] = new_state[r][c-1], new_state[r][c]
        elif op == 'right' and c < 2:
            new_state[r][c], new_state[r][c+1] = new_state[r][c+1], new_state[r][c]
        else:
            continue
        expanded_nodes.append(Node(new_state, node, op, node.g + 1))
    return expanded_nodes

# --- Heuristics ---
def misplaced_tile(state, goal):
    count = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0 and state[i][j] != goal[i][j]:
                count += 1
    return count

def manhattan_distance(state, goal):
    distance = 0
    goal_map = {goal[i][j]: (i, j) for i in range(3) for j in range(3)}
    for i in range(3):
        for j in range(3):
            tile = state[i][j]
            if tile != 0:
                goal_r, goal_c = goal_map[tile]
                distance += abs(i - goal_r) + abs(j - goal_c)
    return distance

# --- Driver Search Algorithm ---
def general_search(problem, mode):
    # nodes = MAKE-QUEUE(MAKE-NODE(problem.INITIAL-STATE))
    initial_h = 0
    if mode == 2: initial_h = misplaced_tile(problem.INITIAL_STATE, problem.GOAL_STATE)
    if mode == 3: initial_h = manhattan_distance(problem.INITIAL_STATE, problem.GOAL_STATE)
    
    start_node = Node(problem.INITIAL_STATE, h=initial_h)
    nodes = [start_node] # The priority queue
    heapq.heapify(nodes)
    
    explored = set()
    max_queue_size = 1
    nodes_expanded = 0
    
    print(f"Expanding state with g(n) = {start_node.g} and h(n) = {start_node.h}:")
    print_puzzle(start_node.STATE)

    while nodes:
        max_queue_size = max(max_queue_size, len(nodes))
        node = heapq.heappop(nodes) # REMOVE-FRONT(nodes)
        
        if problem.GOAL_TEST(node.STATE):
            print("\nGoal state!")
            print(f"Solution depth was {node.g}")
            print(f"Number of nodes expanded: {nodes_expanded}")
            print(f"Max queue size: {max_queue_size}")
            return node
        
        state_tuple = tuple(tuple(row) for row in node.STATE)
        if state_tuple in explored:
            continue
        explored.add(state_tuple)
        
        nodes_expanded += 1
        successors = EXPAND(node, problem.OPERATORS)
        
        for child in successors:
            if mode == 1: child.h = 0 # Uniform Cost
            elif mode == 2: child.h = misplaced_tile(child.STATE, problem.GOAL_STATE)
            elif mode == 3: child.h = manhattan_distance(child.STATE, problem.GOAL_STATE)
            child.f = child.g + child.h
            heapq.heappush(nodes, child)

    return "failure"

def print_puzzle(state):
    for row in state:
        print(row)

if __name__ == "__main__":
    print("Welcome to my 8-puzzle solver.")
    puzzle_choice = input("Type '1' to use a default puzzle, or '2' to enter your own: ")
    if puzzle_choice == '1':
        initial_state = [[1, 2, 4],
                         [5, 8, 0],
                         [7, 3, 6]]
    else:
        print("Enter your puzzle, using a zero for the blank. Space delimited numbers.")
        Row_1 = list(map(int, input("Enter first row: ").split()))
        Row_2 = list(map(int, input("Enter second row: ").split()))
        Row_3 = list(map(int, input("Enter third row: ").split()))
        initial_state = [Row_1, Row_2, Row_3]

    print("Enter choice of algorithm:\n1. Uniform Cost Search\n2. A* with Misplaced Tile\n3. A* with Manhattan Distance")
    algo = int(input())
    prob = Problem(initial_state)
    general_search(prob, algo)

Welcome to my 8-puzzle solver.


Type '1' to use a default puzzle, or '2' to enter your own:  1


Enter choice of algorithm:
1. Uniform Cost Search
2. A* with Misplaced Tile
3. A* with Manhattan Distance


 2


Expanding state with g(n) = 0 and h(n) = 5:
[1, 2, 4]
[5, 8, 0]
[7, 3, 6]
